# ClockSandbox — Phase I Smoke Test

- Ideal vs Noisy (white fractional-frequency) over 1 day.
- Deterministic seed.
- Plots of readings and difference, plus basic metrics.

In [ ]:
from src.clocks.ideal import IdealClock
from src.clocks.noisy import NoisyOscillatorClock
from src.analysis import run_clocks, compare_clocks, plot_comparison
ideal = IdealClock()
noisy = NoisyOscillatorClock(sigma_y=1e-11, seed=42)
ts = run_clocks([ideal, noisy], duration=86_400.0, dt=1.0)
metrics = compare_clocks(ts["time"], ts["clock_0"], ts["clock_1"])
metrics

In [ ]:
# Plot (disable if running headless)
plot_comparison(ts, labels=["Ideal", "Noisy (white freq)"])
